# Assignment 2

In [1]:
import os
os.chdir('/home/ubuntu/fastai/courses/dl1/')

In [2]:
# Import Library
import numpy as np
from functools import partial
import pandas as pd

import dill as pickle
import torchtext
import torch
from torch import nn
from torch.nn import functional as F
import spacy
import fastai.nlp
from fastai.learner import fit
from fastai.core import V
from fastai.metrics import accuracy
from fastai.lm_rnn import seq2seq_reg, repackage_var
from fastai.nlp import RNN_Learner
from IPython.display import display
from IPython.lib.display import FileLink

from pathlib import Path

/home/ubuntu/src/anaconda3/envs/fastai/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/ubuntu/src/anaconda3/envs/fastai/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/ubuntu/src/anaconda3/envs/fastai/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/ubuntu/src/anaconda3/envs/fastai/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192, got 176
  return f(*args, **kwds)
/home/ubuntu/src/anaconda3/envs/fastai/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 

In [3]:
df_train = pd.read_csv('./data/ass2/train.csv')

In [4]:
len(df_train)

10387

In [5]:
df_train.head()

,id,text,airline,tweet_location,user_timezone,sentiment
0,0,@JetBlue great flight! Great view! :-) http://...,Delta,NaN,NaN,positive
1,1,"@united they're not, actually. gate agent was ...",United,chicago,NaN,negative
2,2,@AmericanAir No worries they called back 4 hrs...,American,"Dallas, Texas",NaN,negative
3,3,@united thank you. There was one here a few mo...,United,"New York, NY",America/New_York,positive
4,4,@united Brothers luggage was lost on Copa Airl...,United,"Kearney, Nebraska",Central Time (US & Canada),negative


In [6]:
df_test = pd.read_csv('./data/ass2/test.csv')

In [7]:
df_test.head()

,id,text,airline,tweet_location,user_timezone
0,0,@USAirways if one with @AmericanAir why can't ...,US Airways,NaN,NaN
1,1,@VirginAmerica You'd think paying an extra $10...,Virgin America,San Diego,Alaska
2,2,"@united according to your DMs, I'm not owed a ...",United,Nottingham,London
3,3,"@USAirways booked an award ticket recently, no...",US Airways,USA,Eastern Time (US & Canada)
4,4,@JetBlue Awesome! #bestairlineever,Delta,NYC,Quito


### Explore data

In [8]:
df_train.sentiment.value_counts(dropna=False)

negative    8242
positive    2145
Name: sentiment, dtype: int64

In [9]:
df_train = df_train.dropna()

In [10]:
df_train = df_train.sample(frac=1, random_state=42)

In [11]:
train_df = df_train[:-1000]
val_df = df_train[-1000:]

In [12]:
print("training size ",train_df.shape)
print("training size ",val_df.shape)

training size  (4422, 6)
training size  (1000, 6)


In [13]:
print("Check training data")
print(train_df.sentiment.value_counts())

Check training data
negative    3408
positive    1014
Name: sentiment, dtype: int64


In [14]:
print("Check validation data")
print(val_df.sentiment.value_counts())

Check validation data
negative    769
positive    231
Name: sentiment, dtype: int64


In [15]:
class DataFrameDataset(torchtext.data.Dataset):

    def __init__(self, df, text_field, label_field, is_test=False, **kwargs):
        fields = [('text', text_field), ('label', label_field)]
        examples = []
        for i, row in df.iterrows():
            label = row.sentiment if not is_test else None
            text = row.text
            examples.append(torchtext.data.Example.fromlist([text, label], fields))

        super().__init__(examples, fields, **kwargs)

    @staticmethod
    def sort_key(ex):
        return len(ex.text)

    @classmethod
    def splits(cls, text_field, label_field, train_df, val_df=None, test_df=None, **kwargs):
        train_data, val_data, test_data = (None, None, None)

        if train_df is not None:
            train_data = cls(train_df.copy(), text_field, label_field, **kwargs)
        if val_df is not None:
            val_data = cls(val_df.copy(), text_field, label_field, **kwargs)
        if test_df is not None:
            test_data = cls(test_df.copy(), text_field, label_field, True, **kwargs)

        return tuple(d for d in (train_data, val_data, test_data) if d is not None)

In [16]:
TEXT_FIELD = torchtext.data.Field(lower=True, tokenize="spacy")
LABEL_FIELD = torchtext.data.Field(sequential=False)

/home/ubuntu/src/anaconda3/envs/fastai/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/ubuntu/src/anaconda3/envs/fastai/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192, got 176
  return f(*args, **kwds)


In [40]:
train_ds, val_ds, test_ds = DataFrameDataset.splits(text_field=TEXT_FIELD, 
                                                    label_field=LABEL_FIELD, train_df=train_df, val_df=val_df, test_df=df_test)

In [41]:
TEXT_FIELD.build_vocab(train_ds)

In [42]:
TEXT_FIELD.vocab

In [43]:
LABEL_FIELD.build_vocab(train_ds)

In [44]:
LABEL_FIELD.vocab.stoi

defaultdict(<function torchtext.vocab._default_unk_index()>,
            {'<unk>': 0, 'negative': 1, 'positive': 2})

In [45]:
next(iter(train_ds)).text

['@americanair',
 'why',
 'am',
 'i',
 'continually',
 'getting',
 'put',
 'on',
 'hold',
 'by',
 'painfully',
 'inexperienced',
 'people',
 'when',
 'calling',
 'your',
 'platinum',
 'desk',
 '?',
 '!']

In [46]:
train_iter, val_iter, test_iter = torchtext.data.BucketIterator.splits(
    (train_ds, val_ds, test_ds), batch_sizes=(16, 256, 256), sort_key=lambda x: len(x.text))

In [47]:
train_batchtrain_ba  = next(iter(train_iter))

In [48]:
train_batchtrain_ba .text.shape

torch.Size([32, 16])

In [49]:
PATH = Path('./data/global-sentiment')
PATH.mkdir(exist_ok=True)
bs = 16                        

In [50]:
text_data = fastai.nlp.TextData.from_splits(PATH, (train_ds, val_ds, test_ds), bs=bs, text_name='text', label_name='label')

In [51]:
train_batch, y = next(iter(text_data.trn_dl))

In [52]:
text_data.trn_dl.src

In [53]:
train_batch.shape

torch.Size([32, 16])

In [54]:
train_batch, y = next(iter(text_data.trn_dl))

In [55]:
train_batch.shape

torch.Size([32, 16])

# Using RNN

In [77]:
bs = 64
bptt = 70
em_sz = 200
nh = 500
nl = 3

In [78]:
opt_fn = partial(torch.optim.Adam, betas=(0.7, 0.99))
learner = text_data.get_model(opt_fn, 1500, bptt, emb_sz=em_sz,n_hid=nh, n_layers=nl,
                             dropout=0.1, dropouti=0.4,wdrop=0.5, dropoute=0.05, dropouth=0.3)

In [79]:
learner

SequentialRNN(
  (0): MultiBatchRNN(
    (encoder): Embedding(7748, 200, padding_idx=1)
    (encoder_with_dropout): EmbeddingDropout(
      (embed): Embedding(7748, 200, padding_idx=1)
    )
    (rnns): ModuleList(
      (0): WeightDrop(
        (module): LSTM(200, 500)
      )
      (1): WeightDrop(
        (module): LSTM(500, 500)
      )
      (2): WeightDrop(
        (module): LSTM(500, 200)
      )
    )
    (dropouti): LockedDropout(
    )
    (dropouths): ModuleList(
      (0): LockedDropout(
      )
      (1): LockedDropout(
      )
      (2): LockedDropout(
      )
    )
  )
  (1): PoolingLinearClassifier(
    (layers): ModuleList(
      (0): LinearBlock(
        (lin): Linear(in_features=600, out_features=3, bias=True)
        (drop): Dropout(p=0.1)
        (bn): BatchNorm1d(600, eps=1e-05, momentum=0.1, affine=True)
      )
    )
  )
)

In [80]:
learner.reg_fn =partial(seq2seq_reg, alpha=2, beta=1)
learner.clip=0.3

In [81]:
learner.fit(3e-3, 4, wds=1e-6, cycle_len=1, cycle_mult=2, metrics=[accuracy])

epoch      trn_loss   val_loss   accuracy                    
    0      0.399485   0.387968   0.848259  
    1      0.341437   0.275574   0.885938                    
    2      0.205169   0.281269   0.889286                    
    3      0.227581   0.245306   0.903929                    
    4      0.14264    0.237205   0.903125                    
    5      0.075634   0.289302   0.904464                     
    6      0.073863   0.32075    0.902188                     
    7      0.139736   0.307574   0.90567                      
    8      0.09699    0.350185   0.899375                     
    9      0.061942   0.329287   0.905357                     
    10     0.059824   0.350862   0.913839                     
    11     0.040136   0.401867   0.914821                     
    12     0.039077   0.378826   0.907768                     
    13     0.033449   0.410071   0.908929                     
    14     0.037424   0.418922   0.910223                     



[array([0.41892]), 0.9102232142857143]

In [82]:
learner.save_encoder('enc1')

In [83]:
learner.save('model_adam1')

### Run on Test Data

In [84]:
df_test.head()

,id,text,airline,tweet_location,user_timezone
0,0,@USAirways if one with @AmericanAir why can't ...,US Airways,NaN,NaN
1,1,@VirginAmerica You'd think paying an extra $10...,Virgin America,San Diego,Alaska
2,2,"@united according to your DMs, I'm not owed a ...",United,Nottingham,London
3,3,"@USAirways booked an award ticket recently, no...",US Airways,USA,Eastern Time (US & Canada)
4,4,@JetBlue Awesome! #bestairlineever,Delta,NYC,Quito


In [107]:
learner.data.test_dl.src.sort = False
learner.data.test_dl.src.sort_within_batch = False
learner.data.test_dl.src.shuffle = False

In [108]:
probs = learner.predict(is_test=True,)

In [109]:
preds = np.argmax(probs, axis=1)

In [110]:
PATH = "data/ass2"
SUBM = f'{PATH}/subm/'
os.makedirs(SUBM, exist_ok=True)

In [111]:
pd.DataFrame({
    'id': df_test['id'],
    'sentiment': [LABEL_FIELD.vocab.itos[p] for p in preds]}).to_csv(f'{SUBM}sub_ass2.csv', index=False)

In [112]:
FileLink(f'{SUBM}sub_ass2.csv')

/home/ubuntu/fastai/courses/dl1/data/ass2/subm/sub_ass2.csv

In [113]:
!kaggle competitions submit -c high-flyers -f data/ass2/subm/sub_ass2.csv -m "Ngo Duy Vu Submission"

Successfully submitted to High Flyers

In [105]:
!pip install update kaggle

# Improving Model

In [1]:
#https://www.kaggle.com/crowdflower/first-gop-debate-twitter-sentiment
#https://www.kaggle.com/crowdflower/twitter-airline-sentiment
#wget http://cs.stanford.edu/people/alecmgo/trainingandtestdata.zip --no-check-certificate

In [17]:
SF_df = pd.read_csv('./data/datamining/stanford/training.1600000.processed.noemoticon.csv', encoding="ISO-8859-1", header = None)
SF_df.head(10)

,0,1,2,3,4,5
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
5,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew
6,0,1467811592,Mon Apr 06 22:20:03 PDT 2009,NO_QUERY,mybirch,Need a hug
7,0,1467811594,Mon Apr 06 22:20:03 PDT 2009,NO_QUERY,coZZ,@LOLTrish hey long time no see! Yes.. Rains a...
8,0,1467811795,Mon Apr 06 22:20:05 PDT 2009,NO_QUERY,2Hood4Hollywood,@Tatiana_K nope they didn't have it
9,0,1467812025,Mon Apr 06 22:20:09 PDT 2009,NO_QUERY,mimismo,@twittera que me muera ?


In [18]:
SF_df = SF_df.loc[SF_df[0].isin(['0','4'])]
SF_df = SF_df[[0, 5]]
SF_df.rename(columns={0: 'sentiment', 5: 'text'}, inplace=True)

In [19]:
sentiment = []
for i in SF_df.sentiment:
    if i == 0:
        sentiment.append('negative')
    else:
        sentiment.append('positive')
        
SF_df['sentiment'] = sentiment

In [20]:
print(SF_df.shape[0])
print(SF_df.sentiment.value_counts(dropna=False))
SF_df.head(10)

1600000
negative    800000
positive    800000
Name: sentiment, dtype: int64


,sentiment,text
0,negative,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,negative,is upset that he can't update his Facebook by ...
2,negative,@Kenichan I dived many times for the ball. Man...
3,negative,my whole body feels itchy and like its on fire
4,negative,"@nationwideclass no, it's not behaving at all...."
5,negative,@Kwesidei not the whole crew
6,negative,Need a hug
7,negative,@LOLTrish hey long time no see! Yes.. Rains a...
8,negative,@Tatiana_K nope they didn't have it
9,negative,@twittera que me muera ?


In [21]:
GOP_df = pd.read_csv('./data/datamining/GOP/Sentiment.csv')
GOP_df = GOP_df.loc[GOP_df['sentiment'].isin(["Positive","Negative"])]
GOP_df = GOP_df[['text', 'sentiment']]
print(GOP_df.shape[0])
GOP_df.head(10)

10729


,text,sentiment
1,RT @ScottWalker: Didn't catch the full #GOPdeb...,Positive
3,RT @RobGeorge: That Carly Fiorina is trending ...,Positive
4,RT @DanScavino: #GOPDebate w/ @realDonaldTrump...,Positive
5,"RT @GregAbbott_TX: @TedCruz: ""On my first day ...",Positive
6,RT @warriorwoman91: I liked her and was happy ...,Negative
8,Deer in the headlights RT @lizzwinstead: Ben C...,Negative
9,RT @NancyOsborne180: Last night's debate prove...,Negative
10,@JGreenDC @realDonaldTrump In all fairness #Bi...,Negative
11,RT @WayneDupreeShow: Just woke up to tweet thi...,Positive
12,Me reading my family's comments about how grea...,Negative


In [22]:
AL_df = pd.read_csv('./data/datamining/airline/Tweets.csv')
AL_df = AL_df.loc[AL_df['airline_sentiment'].isin(["positive","negative"])]
AL_df = AL_df[['text', 'airline_sentiment']]
AL_df.rename(columns={'airline_sentiment': 'sentiment'}, inplace=True)
print(AL_df.shape[0])
AL_df.head(10)

11541


,text,sentiment
1,@VirginAmerica plus you've added commercials t...,positive
3,@VirginAmerica it's really aggressive to blast...,negative
4,@VirginAmerica and it's a really big bad thing...,negative
5,@VirginAmerica seriously would pay $30 a fligh...,negative
6,"@VirginAmerica yes, nearly every time I fly VX...",positive
8,"@virginamerica Well, I didn't…but NOW I DO! :-D",positive
9,"@VirginAmerica it was amazing, and arrived an ...",positive
11,@VirginAmerica I &lt;3 pretty graphics. so muc...,positive
12,@VirginAmerica This is such a great deal! Alre...,positive
13,@VirginAmerica @virginmedia I'm flying your #f...,positive


In [23]:
# Remove unused features of dataset
df_train = df_train[['text', 'sentiment']]
print(df_train.shape[0])
df_train.head(10)

5422


,text,sentiment
10095,@AmericanAir why am I continually getting put ...,negative
9986,@USAirways not happy!! Trying to get home on C...,negative
6117,"@united so you told me to go, knowing what the...",negative
7727,"@AmericanAir Also, I have to wait more than 2 ...",negative
3044,Thanks @united for writing back. To assist you...,negative
8628,@VirginAmerica Thanks for a great flight from ...,positive
8705,@USAirways Everyone on Flight 669 from LAX to ...,negative
4104,@united ours in July. You have ZERO excuses fo...,negative
8847,@united we needed them here asap. Will they ma...,negative
5522,@VirginAmerica sad to learn you no longer fly ...,negative


In [24]:
# Combine all data together
df_train = pd.concat([df_train, GOP_df, AL_df, SF_df], axis=0)
print(df_train.shape[0])
df_train.head(10)

1627692


/home/ubuntu/src/anaconda3/envs/fastai/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


,sentiment,text
10095,negative,@AmericanAir why am I continually getting put ...
9986,negative,@USAirways not happy!! Trying to get home on C...
6117,negative,"@united so you told me to go, knowing what the..."
7727,negative,"@AmericanAir Also, I have to wait more than 2 ..."
3044,negative,Thanks @united for writing back. To assist you...
8628,positive,@VirginAmerica Thanks for a great flight from ...
8705,negative,@USAirways Everyone on Flight 669 from LAX to ...
4104,negative,@united ours in July. You have ZERO excuses fo...
8847,negative,@united we needed them here asap. Will they ma...
5522,negative,@VirginAmerica sad to learn you no longer fly ...


In [25]:
df_train.sentiment = df_train.sentiment.replace({'Positive': 'positive', 'Negative': 'negative'})
df_train.sentiment.value_counts(dropna=False)

negative    821848
positive    805844
Name: sentiment, dtype: int64

In [26]:
## Regular expression
import re
from bs4 import BeautifulSoup
from nltk.tokenize import WordPunctTokenizer

def tweet_cleaner(text):
    tok = WordPunctTokenizer()
    remove_mention = r'@[A-Za-z0-9]+'
    remove_http = r'https?://[A-Za-z0-9./]+'
    combined= r'|'.join((remove_mention, remove_http))
    stripped = re.sub(combined, '', text)
    # Replace all unregconise characters
    try:
        clean = stripped.decode("utf-8-sig").replace(u"\ufffd", "?")
    except:
        clean = stripped
    letters_only = re.sub("[^a-zA-Z]", " ", clean)
    lower_case = letters_only.lower()
    # Join together to remove unneccessary white spaces
    words = tok.tokenize(lower_case)
    return (" ".join(words)).strip()

/home/ubuntu/src/anaconda3/envs/fastai/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [27]:
# Clean data
clean_text = []
for i in df_train['text']:
    clean_text.append(tweet_cleaner(i))

In [28]:
df_train['text'] = clean_text

In [29]:
df_train.head(10)

,sentiment,text
10095,negative,why am i continually getting put on hold by pa...
9986,negative,not happy trying to get home on cancelled flig...
6117,negative,so you told me to go knowing what the issue wa...
7727,negative,also i have to wait more than hours before i c...
3044,negative,thanks for writing back to assist you can retu...
8628,positive,thanks for a great flight from la to boston pi...
8705,negative,everyone on flight from lax to rdu enjoyed wai...
4104,negative,ours in july you have zero excuses for this yo...
8847,negative,we needed them here asap will they make it on ...
5522,negative,sad to learn you no longer fly sfo gt phl hope...


In [30]:
df_train = df_train.sample(frac=1, random_state=42)
train_df = df_train[:-5000]
val_df = df_train[-5000:]

In [31]:
train_ds, val_ds, test_ds = DataFrameDataset.splits(text_field=TEXT_FIELD, 
                                                    label_field=LABEL_FIELD, train_df=train_df, val_df=val_df, test_df=df_test)

In [32]:
TEXT_FIELD.build_vocab(train_ds)

In [33]:
LABEL_FIELD.build_vocab(train_ds)

In [34]:
train_iter, val_iter, test_iter = torchtext.data.BucketIterator.splits(
    (train_ds, val_ds, test_ds), batch_sizes=(16, 256, 256), sort_key=lambda x: len(x.text))

In [35]:
train_batchtrain_ba  = next(iter(train_iter))
PATH = Path('./data/global-sentiment')
PATH.mkdir(exist_ok=True)
bs = 16   

In [36]:
text_data = fastai.nlp.TextData.from_splits(PATH, (train_ds, val_ds, test_ds), bs=bs, text_name='text', label_name='label')

In [37]:
train_batch, y = next(iter(text_data.trn_dl))

In [38]:
bs = 64
bptt = 70
em_sz = 200
nh = 500
nl = 3

In [39]:
opt_fn = partial(torch.optim.Adam, betas=(0.7, 0.99))
learner = text_data.get_model(opt_fn, 1500, bptt, emb_sz=em_sz,n_hid=nh, n_layers=nl,
                             dropout=0.1, dropouti=0.4,wdrop=0.5, dropoute=0.05, dropouth=0.3)

In [40]:
learner.reg_fn =partial(seq2seq_reg, alpha=2, beta=1)
learner.clip=0.3

In [ ]:
learner.fit(3e-3, 4, wds=1e-6, cycle_len=1, cycle_mult=2, metrics=[accuracy])

  1%|          | 529/101419 [12:37<40:08:32,  1.43s/it, loss=0.742]

In [ ]:
learner.data.test_dl.src.sort = False
learner.data.test_dl.src.sort_within_batch = False
learner.data.test_dl.src.shuffle = False

In [ ]:
probs = learner.predict(is_test=True,)

In [ ]:
preds = np.argmax(probs, axis=1)

In [ ]:
PATH = "data/ass2"
SUBM = f'{PATH}/subm/'
os.makedirs(SUBM, exist_ok=True)

In [ ]:
pd.DataFrame({
    'id': df_test['id'],
    'sentiment': [LABEL_FIELD.vocab.itos[p] for p in preds]}).to_csv(f'{SUBM}sub_ass2.csv', index=False)

In [ ]:
!kaggle competitions submit -c high-flyers -f data/ass2/subm/sub_ass2.csv -m "Ngo Duy Vu Submission"

In [ ]:
learner.save_encoder('enc1')
learner.save('model_adam1')

In [ ]:
!sudo shutdown -h now